<a href="https://colab.research.google.com/github/enps2015/simulator_Telos_nivel0/blob/main/SimulatorT%C3%A9losN%C3%ADvel0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1 - Importando as bibliotecas "sqlite3", que permte interagir com banco de dados SQLite, e a biblioteca "pandas" para realizar a manipulação e análise dos dados em Python

In [7]:
import sqlite3
import pandas as pd


2 - Criando o Banco de Dados e as Tabelas

  - Criando a conexão com o banco de dados "empresa_abc.db" (O cursor é um objeto que permite executar comandos SQL no banco de dados)


In [8]:
conn = sqlite3.connect('empresa_abc.db')
cursor = conn.cursor()

 - Criando as tabelas no Python

In [9]:
cursor.execute('''
    CREATE TABLE Clientes (
        id_Cliente INTEGER PRIMARY KEY,
        Nome TEXT,
        Pais TEXT
    )
''')

cursor.execute('''
    CREATE TABLE Produtos (
        id_Produto INTEGER PRIMARY KEY,
        Nome_Produto TEXT,
        Categoria TEXT,
        Preco REAL
    )
''')

cursor.execute('''
    CREATE TABLE Pedidos (
        id_Pedido INTEGER PRIMARY KEY,
        id_Cliente INTEGER,
        Data_Pedido DATE,
        FOREIGN KEY (id_Cliente) REFERENCES Clientes(id_Cliente)
    )
''')

cursor.execute ('''
    CREATE TABLE Detalhes_Pedidos (
        id_Pedido INTEGER,
        id_Produto INTEGER,
        Quantidade INTEGER,
        FOREIGN KEY (id_Pedido) REFERENCES Pedidos(id_Pedido),
        FOREIGN KEY (id_Produto) REFERENCES Produtos(id_Produto)
    )
''')

conn.commit()

3 - Gerando mock de dados ficitícios com Gemini

  Os seguintes prompts foram usados para gerar todas as bases de dados com a genAI:

   - Clientes:

  Crie um mock de dados fictício com 50 linhas para a tabela Clientes, incluindo as colunas ID_Cliente (inteiro), Nome (texto) e Pais (texto).

  Exemplo:

  ID_Cliente | Nome | Pais

 1 | João Silva | Brasil

 2 | Maria Santos | Argentina

- Produtos:

  Crie um mock de dados fictício com 50 linhas para a tabela Produtos, incluindo as colunas ID_Produto (inteiro), Nome_Produto (texto), Categoria (texto) e Preco (número decimal).

  Exemplo:

  ID_Produto | Nome_Produto | Categoria | Preco

  1 | Smartphone X | Eletrônicos | 1299.99

  2 | Camiseta Manga Curta | Vestuário | 49.90

  - Pedidos:

  Crie um mock de dados fictício com 50 linhas para a tabela Pedidos, incluindo as colunas ID_Pedido (inteiro), ID_Cliente (inteiro, entre 1 e 100), Data_Pedido (data no formato AAAAMMDD).

  Exemplo:

  ID_Pedido | ID_Cliente | Data_Pedido

  1 | 1 | 20231201

  2 | 3 | 20231205

  - Detalhes_Pedido:

  Crie um mock de dados fictício com 50 linhas para a tabela Detalhes_Pedido, incluindo as colunas ID_Pedido (inteiro, entre 1 e 100), ID_Produto (inteiro, entre 1 e 100) e Quantidade (inteiro).

  Exemplo:

  ID_Pedido | ID_Produto | Quantidade

  1 | 1 | 2

  2 | 3 | 1

Como resultado foram gerados os arquivos .csv:
  - clientes.csv
  - Produtos.csv
  - Pedidos.csv
  - detalhesPedido.csv


4 - Inserindo os dados no Banco de Dados empresa_abc.db

  - Crinado conexão com o banco de dados

  - Inserindo os dados com a biblioteca Pandas

In [10]:
# Inserindo dados
# Lendo dados do CSV
clientes_df = pd.read_csv('clientes.csv')

# Inserindo dados na tabela Clientes
for index, row in clientes_df.iterrows():
    cursor.execute("INSERT INTO Clientes (id_Cliente, Nome, Pais) VALUES (?, ?, ?)", (row['id_Cliente'], row['Nome'], row['Pais']))

# Lendo dados do CSV
produtos_df = pd.read_csv('Produtos.csv')

# Inserindo dados na tabela Produtos
for index, row in produtos_df.iterrows():
    cursor.execute("INSERT INTO Produtos (id_Produto, Nome_Produto, Categoria, Preco) VALUES (?, ?, ?, ?)", (row['id_Produto'], row['Nome_Produto'], row['Categoria'], row['Preco']))

# Lendo dados do CSV
pedidos_df = pd.read_csv('Pedidos.csv')

# Convertendo a coluna Data_Pedido para o tipo datetime (usando o formato correto)
pedidos_df['Data_Pedido'] = pd.to_datetime(pedidos_df['Data_Pedido'], format='%Y%m%d')

# Inserindo dados na tabela Pedidos
for index, row in pedidos_df.iterrows():
    cursor.execute("INSERT INTO Pedidos (id_Pedido, id_Cliente, Data_Pedido) VALUES (?, ?, ?)",
                   (row['id_Pedido'], row['id_Cliente'], row['Data_Pedido'].strftime('%Y-%m-%d')))

# Inserir dados na tabela Detalhes_Pedidos (convertendo para int diretamente)
detalhesPedido_df = pd.read_csv('detalhesPedido.csv')

for index, row in detalhesPedido_df.iterrows():
    cursor.execute("INSERT INTO Detalhes_Pedidos (id_Pedido, id_Produto, Quantidade) VALUES (?, ?, ?)",
                   (int(row['id_Pedido']), int(row['id_Produto']), int(row['Quantidade'])))

conn.commit()


5 - Analisando Dados com Pandas e respondendo Perguntas de Negócio

  - Análise de Clientes

  a -  Quais são os top 5 países em número de clientes?


In [11]:
# Carregar os dados da tabela Clientes
clientes_df = pd.read_sql_query("SELECT * FROM Clientes", conn)

# Agrupar por país e contar o número de clientes
top_paises_df = clientes_df.groupby('Pais')['id_Cliente'].count().sort_values(ascending=False).head(5).reset_index()

# Mostrar o resultado
print("Top 5 Países em número de clientes:")
print(top_paises_df)

Top 5 Países em número de clientes:
        Pais  id_Cliente
0     Brasil          25
1  Argentina          13
2   Portugal          12


  - Análise de Clientes
  
    b - Quantos clientes únicos realizam mais de um pedido?

In [12]:
# Carregar os dados das tabelas
clientes_df = pd.read_sql_query("SELECT * FROM Clientes", conn)
pedidos_df = pd.read_sql_query("SELECT * FROM Pedidos", conn)
detalhes_pedido_df = pd.read_sql_query("SELECT * FROM Detalhes_Pedidos", conn)

# Juntar as tabelas
merged_df = clientes_df.merge(pedidos_df, on='id_Cliente').merge(detalhes_pedido_df, on='id_Pedido')

# Agrupar por id_Cliente e contar o número de pedidos
pedidos_por_cliente_df = merged_df.groupby('id_Cliente')['id_Pedido'].count()

# Filtrar clientes com mais de um pedido
clientes_com_mais_de_um_pedido_df = pedidos_por_cliente_df[pedidos_por_cliente_df > 1]

# Contar o número de clientes únicos
num_clientes_unicos = len(clientes_com_mais_de_um_pedido_df.index)

print(f"Número de clientes únicos que realizaram mais de um pedido: {num_clientes_unicos}")

Número de clientes únicos que realizaram mais de um pedido: 5


  - Análise de Clientes
  
    c - Qual é o valor médio de pedidos por cliente em cada país?

In [17]:
# Carregar os dados das tabelas
clientes_df = pd.read_sql_query("SELECT * FROM Clientes", conn)
pedidos_df = pd.read_sql_query("SELECT * FROM Pedidos", conn)
detalhes_pedido_df = pd.read_sql_query("SELECT * FROM Detalhes_Pedidos", conn)
produtos_df = pd.read_sql_query("SELECT * FROM Produtos", conn)

# Juntar as tabelas usando id_Pedido e id_Cliente
merged_df = pedidos_df.merge(detalhes_pedido_df, on='id_Pedido').merge(produtos_df, on='id_Produto').merge(clientes_df, on='id_Cliente')

# Calcular o valor total do pedido
merged_df['Valor_Total'] = merged_df['Quantidade'] * merged_df['Preco']

# Agrupar por id_Cliente e Pais e somar o valor total
valor_total_por_cliente_pais_df = merged_df.groupby(['id_Cliente', 'Pais'])['Valor_Total'].sum().reset_index()

# Calcular o valor médio por cliente e país
valor_medio_por_cliente_pais_df = valor_total_por_cliente_pais_df.groupby('Pais')['Valor_Total'].mean().reset_index()

print("Valor médio de pedidos por cliente em cada país:")
print(valor_medio_por_cliente_pais_df)

Valor médio de pedidos por cliente em cada país:
        Pais  Valor_Total
0  Argentina     42978.29
1     Brasil     28144.87
2   Portugal     27958.42


  - Análise de Clientes
  
    d - Quais são os 3 principais clientes em termos de valor total de pedidos?

In [19]:
# Carregar os dados das tabelas
clientes_df = pd.read_sql_query("SELECT * FROM Clientes", conn)
detalhes_pedido_df = pd.read_sql_query("SELECT * FROM Detalhes_Pedidos", conn)
produtos_df = pd.read_sql_query("SELECT * FROM Produtos", conn)
pedidos_df = pd.read_sql_query("SELECT * FROM Pedidos", conn)

# Juntar as tabelas
merged_df = pedidos_df.merge(detalhes_pedido_df, on='id_Pedido').merge(produtos_df, on='id_Produto')

# Adicionar a coluna id_Cliente ao DataFrame merged_df
merged_df = merged_df.merge(clientes_df, on='id_Cliente')

# Calcular o valor total do pedido
merged_df['Valor_Total'] = merged_df['Quantidade'] * merged_df['Preco']

# Agrupar por id_Cliente e somar o valor total
valor_total_por_cliente_df = merged_df.groupby('id_Cliente')['Valor_Total'].sum().reset_index()

# Ordenar os clientes pelo valor total em ordem decrescente
top_clientes_df = valor_total_por_cliente_df.sort_values('Valor_Total', ascending=False).head(3)

print("Top 3 clientes em termos de valor total de pedidos:")
print(top_clientes_df)

Top 3 clientes em termos de valor total de pedidos:
   id_Cliente  Valor_Total
1           2     42978.29
4           5     34328.29
3           4     31067.92


  - Análise de Produtos
    
    a - Quais são as top 5 categorias de produtos mais vendidas?

In [20]:
import pandas as pd

# Carregar os dados das tabelas
detalhes_pedido_df = pd.read_sql_query("SELECT * FROM Detalhes_Pedidos", conn)
produtos_df = pd.read_sql_query("SELECT * FROM Produtos", conn)

# Juntar as tabelas
merged_df = detalhes_pedido_df.merge(produtos_df, on='id_Produto')

# Agrupar por Categoria e somar a Quantidade
top_categorias_df = merged_df.groupby('Categoria')['Quantidade'].sum().sort_values(ascending=False).head(5).reset_index()

print("Top 5 categorias de produtos mais vendidas:")
print(top_categorias_df)

Top 5 categorias de produtos mais vendidas:
     Categoria  Quantidade
0  Eletrônicos          71
1       Livros          27
2     Calçados          20
3    Vestuário          12


  - Análise de Produtos
    
    b - Qual é o produto mais caro que foi vendido?

In [21]:
import pandas as pd

# Carregar os dados das tabelas
detalhes_pedido_df = pd.read_sql_query("SELECT * FROM Detalhes_Pedidos", conn)
produtos_df = pd.read_sql_query("SELECT * FROM Produtos", conn)

# Juntar as tabelas
merged_df = detalhes_pedido_df.merge(produtos_df, on='id_Produto')

# Encontrar o produto com maior Preco
produto_mais_caro_df = merged_df.sort_values('Preco', ascending=False).head(1)

print("Produto mais caro que foi vendido:")
print(produto_mais_caro_df[['Nome_Produto', 'Preco']])

Produto mais caro que foi vendido:
      Nome_Produto    Preco
13  Notebook Gamer  4999.99


  - Análise de Produtos
    
    c - Qual categoria de produto gera a maior receita

---



In [22]:
import pandas as pd

# Carregar os dados das tabelas
detalhes_pedido_df = pd.read_sql_query("SELECT * FROM Detalhes_Pedidos", conn)
produtos_df = pd.read_sql_query("SELECT * FROM Produtos", conn)

# Juntar as tabelas
merged_df = detalhes_pedido_df.merge(produtos_df, on='id_Produto')

# Calcular a receita
merged_df['Receita'] = merged_df['Quantidade'] * merged_df['Preco']

# Agrupar por Categoria e somar a Receita
categoria_maior_receita_df = merged_df.groupby('Categoria')['Receita'].sum().sort_values(ascending=False).head(1).reset_index()

print("Categoria de produto que gera a maior receita:")
print(categoria_maior_receita_df)

Categoria de produto que gera a maior receita:
     Categoria    Receita
0  Eletrônicos  150697.22


  - Análise de Produtos
    
    d - Quais são os top 3 produtos mais vendidos?


In [23]:
import pandas as pd

# Carregar os dados das tabelas
detalhes_pedido_df = pd.read_sql_query("SELECT * FROM Detalhes_Pedidos", conn)
produtos_df = pd.read_sql_query("SELECT * FROM Produtos", conn)

# Juntar as tabelas
merged_df = detalhes_pedido_df.merge(produtos_df, on='id_Produto')

# Agrupar por id_Produto e somar a Quantidade
top_produtos_df = merged_df.groupby('id_Produto')['Quantidade'].sum().sort_values(ascending=False).head(3).reset_index()

# Adicionar o Nome_Produto
top_produtos_df = top_produtos_df.merge(produtos_df[['id_Produto', 'Nome_Produto']], on='id_Produto')

print("Top 3 produtos mais vendidos:")
print(top_produtos_df[['Nome_Produto', 'Quantidade']])

Top 3 produtos mais vendidos:
                 Nome_Produto  Quantidade
0  Livro de Ficção Científica          27
1                Smartphone X          26
2                  Smartwatch          23


  - Análise de Pedidos
    
    a - 	Qual é o número total de pedidos realizados por mês?
    


In [24]:
import pandas as pd

# Carregar os dados da tabela Pedidos
pedidos_df = pd.read_sql_query("SELECT * FROM Pedidos", conn)

# Converter a coluna Data_Pedido para o tipo datetime (se necessário)
pedidos_df['Data_Pedido'] = pd.to_datetime(pedidos_df['Data_Pedido'])

# Agrupar por mês e contar o número de pedidos
pedidos_por_mes_df = pedidos_df.groupby(pedidos_df['Data_Pedido'].dt.month)['id_Pedido'].count().reset_index()

print("Número total de pedidos realizados por mês:")
print(pedidos_por_mes_df)

Número total de pedidos realizados por mês:
   Data_Pedido  id_Pedido
0            1         15
1            2         15
2            3          5
3           12         15


  - Análise de Pedidos
           
    b - 	Qual é o valor médio de um pedido?

In [25]:
import pandas as pd

# Carregar os dados das tabelas
pedidos_df = pd.read_sql_query("SELECT * FROM Pedidos", conn)
detalhes_pedido_df = pd.read_sql_query("SELECT * FROM Detalhes_Pedidos", conn)
produtos_df = pd.read_sql_query("SELECT * FROM Produtos", conn)

# Juntar as tabelas
merged_df = pedidos_df.merge(detalhes_pedido_df, on='id_Pedido').merge(produtos_df, on='id_Produto')

# Calcular o valor total do pedido
merged_df['Valor_Total'] = merged_df['Quantidade'] * merged_df['Preco']

# Agrupar por id_Pedido e somar o valor total
valor_total_por_pedido_df = merged_df.groupby('id_Pedido')['Valor_Total'].sum().reset_index()

# Calcular o valor médio de um pedido
valor_medio_pedido = valor_total_por_pedido_df['Valor_Total'].mean()

print(f"Valor médio de um pedido: {valor_medio_pedido:.2f}")

Valor médio de um pedido: 3107.43


  - Análise de Pedidos
    
    c -	Quais são os dias da semana com mais pedidos?


In [ ]:
# Em nenhuma das 4 tabelas existe informações que diga respeito aos dias da semana em que foram feitas as compras!

# Dessa forma não temos como achar esse valor com base nos dados apresentados nas 4 tabelas do banco de dados

  - Análise de Pedidos
    
    d - 	Qual é o país com o maior número de pedidos?

In [27]:
import pandas as pd

# Carregar os dados das tabelas
pedidos_df = pd.read_sql_query("SELECT * FROM Pedidos", conn)
clientes_df = pd.read_sql_query("SELECT * FROM Clientes", conn)

# Juntar as tabelas
merged_df = pedidos_df.merge(clientes_df, on='id_Cliente')

# Agrupar por Pais e contar o número de pedidos
pedidos_por_pais_df = merged_df.groupby('Pais')['id_Pedido'].count().sort_values(ascending=False).head(1).reset_index()

print("País com o maior número de pedidos:")
print(pedidos_por_pais_df)

País com o maior número de pedidos:
     Pais  id_Pedido
0  Brasil         30


  - Análise de Tendências de Vendas

    a -	Como as vendas de cada categoria de produto mudaram ao longo do ano?



  - Análise de Tendências de Vendas

    b - 	Existe alguma correlação entre o país do cliente e a categoria de produto comprada?

   

In [29]:
import pandas as pd

# Carregar os dados das tabelas
detalhes_pedido_df = pd.read_sql_query("SELECT * FROM Detalhes_Pedidos", conn)
produtos_df = pd.read_sql_query("SELECT * FROM Produtos", conn)
clientes_df = pd.read_sql_query("SELECT * FROM Clientes", conn)
pedidos_df = pd.read_sql_query("SELECT * FROM Pedidos", conn)

# Juntar as tabelas
merged_df = detalhes_pedido_df.merge(produtos_df, on='id_Produto').merge(pedidos_df, on='id_Pedido').merge(clientes_df, on='id_Cliente')

# Criar uma tabela de frequência para verificar a correlação
tabela_frequencia = pd.crosstab(merged_df['Pais'], merged_df['Categoria'], normalize='index')

print("Correlação entre país do cliente e categoria de produto comprada:")
print(tabela_frequencia)

Correlação entre país do cliente e categoria de produto comprada:
Categoria  Calçados  Eletrônicos  Livros  Vestuário
Pais                                               
Argentina  0.200000          0.6     0.1   0.100000
Brasil     0.166667          0.5     0.2   0.133333
Portugal   0.100000          0.6     0.2   0.100000


  - Análise de Tendências de Vendas

    c - 	Como o valor médio do pedido varia por mês?



In [30]:
import pandas as pd

# Carregar os dados das tabelas
detalhes_pedido_df = pd.read_sql_query("SELECT * FROM Detalhes_Pedidos", conn)
produtos_df = pd.read_sql_query("SELECT * FROM Produtos", conn)
pedidos_df = pd.read_sql_query("SELECT * FROM Pedidos", conn)

# Juntar as tabelas
merged_df = detalhes_pedido_df.merge(produtos_df, on='id_Produto').merge(pedidos_df, on='id_Pedido')

# Calcular o valor total do pedido
merged_df['Valor_Total'] = merged_df['Quantidade'] * merged_df['Preco']

# Converter a coluna Data_Pedido para o tipo datetime
merged_df['Data_Pedido'] = pd.to_datetime(merged_df['Data_Pedido'])

# Agrupar por mês e calcular o valor médio do pedido
valor_medio_pedido_por_mes_df = merged_df.groupby(merged_df['Data_Pedido'].dt.month)['Valor_Total'].mean().reset_index()

print("Valor médio do pedido por mês:")
print(valor_medio_pedido_por_mes_df)

Valor médio do pedido por mês:
   Data_Pedido  Valor_Total
0            1  2507.812000
1            2  3490.496667
2            3  2621.794000
3           12  3485.848000


  - Análise de Tendências de Vendas

    d - 	Qual categoria de produto mostra a maior variação de vendas entre os meses?
